In [1]:
# import required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
%matplotlib inline

# Problem 1: K-Nearest Neighbors

In [2]:
# read data
trees = pd.read_csv('HW5a_nyc_trees.csv', sep=',', names = ['tree_dbh','spc_latin','root_stone','root_grate','root_other','trunk_wire','trnk_light','trnk_other','brch_light','brch_shoe',
                                                            'brch_other','health'], header = None, skiprows = 1)
trees.head()

,tree_dbh,spc_latin,root_stone,root_grate,root_other,trunk_wire,trnk_light,trnk_other,brch_light,brch_shoe,brch_other,health
0,21,Quercus palustris,Yes,No,No,No,No,No,No,No,No,Fair
1,3,Gleditsia triacanthos var. inermis,No,No,No,No,No,No,No,No,No,Good
2,10,Gleditsia triacanthos var. inermis,Yes,No,No,No,No,No,No,No,No,Good
3,11,Gleditsia triacanthos var. inermis,No,No,No,No,No,No,No,No,No,Good
4,11,Gleditsia triacanthos var. inermis,No,No,No,No,No,No,No,No,No,Good


In [3]:
############# PART A ##################
# transform other columns so binary values are transformed to 0 and 1
binary_cols = np.array(['root_stone','root_grate','root_other','trunk_wire','trnk_light','trnk_other','brch_light','brch_shoe','brch_other'])
for i in binary_cols:
    trees[i] = trees[i].map({'Yes': 1, 'No': 0})

# transform latin name and health so categorical values are one-hot encoded
trees = pd.get_dummies(trees, columns = ['spc_latin','health'])
trees.head()


,tree_dbh,root_stone,root_grate,root_other,trunk_wire,trnk_light,trnk_other,brch_light,brch_shoe,brch_other,spc_latin_Gleditsia triacanthos var. inermis,spc_latin_Platanus x acerifolia,spc_latin_Pyrus calleryana,spc_latin_Quercus palustris,health_Fair,health_Good,health_Poor
0,21,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
1,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,10,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,11,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,11,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [24]:
############# PART B ##################
validation_data_length = 25000

# split data into training and validation sets
trees_training = trees.iloc[:-validation_data_length,:]
trees_validation = trees.iloc[-validation_data_length:,:]

# implement k nearest neighbors algorithm using euclidean distance as the distance metric
def knn(k, training_data, validation_data, health_values_int):

    # initialize empty array to store predicted values
    predicted_values = np.zeros([validation_data.shape[0],health_values_int])
    # loop through each row in the validation data
    for i in range(validation_data.shape[0]):
        # calculate euclidean distance between each row in the training data and the current row in the validation data
        diff = training_data.iloc[:,:-health_values_int].values - validation_data.iloc[i,:-health_values_int].values
        euclidean_distance = np.sqrt(np.sum(np.square(diff), axis=1))

        # sort the euclidean distance array and get the indices of the k smallest values
        k_smallest_indices = np.argpartition(euclidean_distance, k)[:k]

        # get the health values of the k nearest neighbors
        k_smallest_health_values = training_data.iloc[k_smallest_indices,-health_values_int:].values

        # get the mode of the k_smallest_health_values array
        test = np.sum(k_smallest_health_values, axis = 0)
        # get index of the maximum value in the test array
        predicted_values[i] = np.zeros(health_values_int, dtype = int)
        predicted_values[i][np.argmax(test)] = 1

    return predicted_values

# test the k nearest neighbors algorithm for different values of k
# k = [1, 10, 100]
k = [10]
health_values_int = 3
for i in k:
    for j in range(25000,25001,25000):
        predicted_values = knn(i, trees_training.iloc[:j,:], trees_validation, health_values_int)
        print('Accuracy for k = ' + str(i) + ' with ' + str(j) + ' training points is ' + str(np.round(100*np.sum(predicted_values == trees_validation.iloc[:,-3:].values)/(health_values_int*trees_validation.shape[0]),3)) + '%')


Accuracy for k = 10 with 25000 training points is 84.485%


In [30]:
# Provide "confusion matrix" for the best k and n
k = 10
# n = 200000
n = 25000
# predicted_values = knn(k, trees_training.iloc[:n,:], trees_validation, health_values_int)
print('Confusion matrix for k = ' + str(k) + ' with ' + str(n) + ' training points is:')

confusion = np.zeros([health_values_int,health_values_int])
k = 0
l = 0
for i in [2,0,1]:
    for j in [2,0,1]:
        confusion[k,l] = np.sum((predicted_values[:,i] == 1) & (trees_validation.iloc[:,-health_values_int:].values[:,j] == 1))
        l += 1
    l = 0
    k += 1

np.set_printoptions(suppress=True)
print(confusion)

Confusion matrix for k = 10 with 25000 training points is:
[[    6.    12.     6.]
 [  143.   352.   671.]
 [  757.  4229. 18824.]]


#### Part B Comments
With a 25000 length validation dataset, my algorithm takes almost 2 minutes to run against just 25000 training points (I comment on possible ways to improve the speed of the algorithm after part C). Therefore, I tested the algorithm with only 2000 validation points against different sizes of training data and different values of k. I determined that k = 10 was one of the better k values, so I ran the algorithm with k = 10 for all 25000 validation points. Part of the reason the algorithm has a fairly high accuracy of ~85% is just because most of the trees have 'good' health, as is shown by the results of the confusion matrix.

In [31]:
############# PART C ##################
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
# implement k nearest neighbors algorithm using sklearn.neighbors.KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
k = [1, 10, 100]
for i in k:
    for j in range(25000,200000,25000):
        knn = KNeighborsClassifier(n_neighbors = i)
        knn.fit(trees_training.iloc[:j,:-3], trees_training.iloc[:j,-3:])
        print('Accuracy for k = ' + str(i) + ' with ' + str(j) + ' training points is ' + str(np.round(100*knn.score(trees_validation.iloc[:,:-3], trees_validation.iloc[:,-3:]),3)) + '%')

Accuracy for k = 1 with 25000 training points is 66.012%
Accuracy for k = 1 with 50000 training points is 68.664%
Accuracy for k = 1 with 75000 training points is 66.784%
Accuracy for k = 1 with 100000 training points is 67.156%
Accuracy for k = 1 with 125000 training points is 65.788%
Accuracy for k = 1 with 150000 training points is 68.848%
Accuracy for k = 1 with 175000 training points is 67.524%
Accuracy for k = 10 with 25000 training points is 73.232%
Accuracy for k = 10 with 50000 training points is 73.98%
Accuracy for k = 10 with 75000 training points is 73.216%
Accuracy for k = 10 with 100000 training points is 73.196%
Accuracy for k = 10 with 125000 training points is 73.8%
Accuracy for k = 10 with 150000 training points is 73.756%
Accuracy for k = 10 with 175000 training points is 72.136%
Accuracy for k = 100 with 25000 training points is 77.78%
Accuracy for k = 100 with 50000 training points is 77.944%
Accuracy for k = 100 with 75000 training points is 77.932%
Accuracy for k

#### Part C Comments
Clearly, the scikit-learn implementation of the KNN algorithm is WAY faster than my algorithm. This algorithm was able to run 20 times with more training points than my algorithm in the same amount of time it took my algorithm to run just once. There's a few things I could do to improve the speed of my algorithm:
1. Use 'csr_matrix' from scikit-learn to turn the training data into a sparse representation because most of the features are 0
2. Use an algorithm like 'Ball Tree' or 'K-D Tree' to find the nearest neighbors much faster. The scikit implementation of KNN uses one of these algorithms which decreases the accuracy of the results slightly but speeds up the search by more than an order of magnitude.

# Problem 2: Spam Text Detection

In [5]:
# read data
messages = pd.read_csv('HW5a_ham_spam.csv', sep=',', names = ['category','message'], header = None, skiprows = 1)
messages.head()

,category,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
